In [1]:
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit,train_test_split


In [2]:
#load the dataset values
dataset = np.load('dataset_numerical_PCA_transformed.npz',allow_pickle = True)
X = dataset['X']
Y_position= dataset['Position']
Y_release_clause = dataset['Release_Clause']


In [4]:
#lets first make stratified splits for training and test data, stratification will be done on the basis of Position 
split = StratifiedShuffleSplit(n_splits = 1,test_size = 0.2,random_state = 42)
for train_index,test_index in split.split(X,Y_position):
    X_train_p,X_test_p = X[train_index],X[test_index]
    Y_train_p,Y_test_p = Y_position[train_index],Y_position[test_index]
    
#for release clause prediction we will used cross validation instead of stratifying the data
from sklearn.model_selection import cross_val_score


In [9]:
#lets train a XGBoost and test it for predicting position zone of player
import xgboost as xgb
model_p = xgb.XGBClassifier()
model_p.fit(X_train_p,Y_train_p,)
y_pred_p=model_p.predict(X_test_p)
accuracy_score(Y_test_p,y_pred_p)

[14:09:18] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8666266145989787

In [10]:
#lets train a Random Forest Classifier
model_RFC = RandomForestClassifier()
model_RFC.fit(X_train_p,Y_train_p)
y_pred_RFC = model_RFC.predict(X_test_p)
accuracy_score(Y_test_p,y_pred_RFC)


/home/abhi/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8525082607389607

In [11]:
#Now lets test a model for regression for predicting release clauses with cross validation scores
model_r = xgb.XGBRegressor()
scores = cross_val_score(model_r,X,Y_release_clause,cv=5)


In [13]:
scores.mean() * -1

1.8517152814960618

quite a decent score, we are off the actual price by just about 2 million euros